In [14]:
import numpy as np
import scipy.stats as sts
import scipy.integrate as ing

def calculateExactQ(tau):
    exactQ = sts.norm.ppf(tau,loc=120,scale=15)
    return exactQ

def int_normal(y):
    f = sts.norm.pdf(y,loc=120,scale=15)
    return y * f

def calculateIntegrale(Q,c,p):
    t1 = p * ing.quad(int_normal,0,Q)[0]   
    t2 = p * Q * sts.norm.sf(Q,loc = 120,scale = 15)
    t3 = c * Q
    return t1 + t2 - t3

def muhatAndSigma2hat(vY):
    muhat = np.mean(vY)
    sigma2hat = np.var(vY)
    return muhat, sigma2hat

def parameterEstimatedQ(tau,muhat,sigmahat):
    estimatedQ = sts.norm.ppf(tau,loc=muhat,scale=sigmahat)
    return estimatedQ

def nonParameterEstimatedQ(tau,vY):
    n = len(vY)
    number = (int(np.ceil(tau * n))) - 1
    vZ = np.sort(vY)
    nonEstQ = vZ[number]
    return nonEstQ

def calcProfitNormal(Q,Y,c,p):
    profit = p * min(Q,Y) - c * Q
    return profit

def calcProfitPNP(Q,c,p,vY):
    sumProfit = 0
    for i in range(len(vY)):
        sumProfit += calcProfitNormal(Q,vY[i],c,p)
    expectedProfit = sumProfit / len(vY)
    return expectedProfit

M = 1000
n = 200
tau = 0.5
c = 1 - tau
p = 1

Qstar = calculateExactQ(tau)
maxExpectedProfit = calculateIntegrale(Qstar,c,p)

parM = np.zeros(M)
nonParM = np.zeros(M)

sumParPLR  = 0
sumNonParPLR = 0

for i in range(M):
    vY = sts.norm.rvs(loc = 120,scale = 15,size = n)
    muhat, sigma2hat = muhatAndSigma2hat(vY)
    sigmahat = np.sqrt(sigma2hat)
    parEstQ = parameterEstimatedQ(tau, muhat, sigmahat)
    nonParEstQ = nonParameterEstimatedQ(tau, vY)
    parExpProfit = calcProfitPNP(parEstQ,c,p,vY)
    nonParExpProfit = calcProfitPNP(nonParEstQ,c,p,vY)
    sumParPLR += np.abs((maxExpectedProfit - parExpProfit) / (maxExpectedProfit))
    sumNonParPLR += np.abs((maxExpectedProfit - nonParExpProfit) / (maxExpectedProfit))
parPLR = np.sqrt( (1 / M) * sumParPLR)
nonParPLR = np.sqrt( (1 / M) * sumNonParPLR)
ratioPLR = nonParPLR / parPLR
print('The PLR-ratio is: ', ratioPLR)



The PLR-ratio is:  0.998305296085282
